In [ ]:
import cellxgene_census
import scanpy as sc
import scvi
import os
import shutil
import numpy as np
import pandas as pd
import anndata as ad
import scipy
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
processedDataPath = "/mnt/iacchus/joe/processed_data/cell_x_gene_atlas/"
rawDataPath = "/mnt/iacchus/joe/raw_data/cell_x_gene_atlas/"

# Data download

In [ ]:
with cellxgene_census.open_soma(census_version="2023-07-25") as census:   
    # Reads SOMADataFrame as a slice
    cell_metadata = census["census_data"]["mus_musculus"].obs.read(
        value_filter = "disease == 'normal' and is_primary_data == False",
        column_names = ["tissue_general"]
    )
    
    # Concatenates results to pyarrow.Table
    cell_metadata = cell_metadata.concat().to_pandas()

In [ ]:
for tissue in cell_metadata.tissue_general.unique():
    with cellxgene_census.open_soma(census_version="2023-07-25") as census:
        print(f"Getting tissue: {tissue}")
        adata = cellxgene_census.get_anndata(
            census = census,
            organism = "Mus musculus",
            obs_value_filter = f"disease == 'normal' and is_primary_data == False and tissue_general == '{tissue}'",
            column_names = {"obs": ["assay", "cell_type", "tissue", "tissue_general", "dataset_id"]},
        )
        adata.write(f"{rawDataPath}{tissue}.h5ad")
        del adata

In [ ]:
files = os.listdir(rawDataPath)
adatas = []
for file in files:
    if not file.startswith("."):
        adatas.append(sc.read(path + file))

adata = ad.concat(adatas, index_unique="-", keys=[i.split(".")[0] for i in files])
adata.var = adatas[0].var
del adatas

In [ ]:
adata.write(f"{processedDataPath}data/cell_x_gene_atlas.h5ad")

# Preprocessing

In [ ]:
adata = sc.read(f"{processedDataPath}data/cell_x_gene_atlas.h5ad")

In [ ]:
gene_info = sc.queries.biomart_annotations("mmusculus", ["ensembl_gene_id", "external_gene_name", "gene_biotype"])
gene_info.index = gene_info.ensembl_gene_id
del gene_info["ensembl_gene_id"]

adata = adata[adata.obs.assay.str.startswith("10x")].copy()
adata.var.index = adata.var.feature_id
adata.var = adata.var.join(gene_info)
adata.var_names = adata.var.feature_name
adata = adata[:, adata.var.gene_biotype == "protein_coding"].copy()
sc.pp.filter_cells(adata, min_genes=500)
sc.pp.filter_genes(adata, min_cells=20)
adata.obs["batch"] = adata.obs.dataset_id.astype("str") + adata.obs.tissue.astype("str")
adata.layers["counts"] = adata.X.copy()
sc.pp.calculate_qc_metrics(adata, inplace=True, log1p=False, percent_top=None)

In [ ]:
adata.obs["batch"] = adata.obs.dataset_id.astype("str") + adata.obs.tissue.astype("str")
adata.layers["counts"] = adata.X.copy()
sc.pp.calculate_qc_metrics(adata, inplace=True, log1p=False, percent_top=None)

In [ ]:
adata.write(f"{processedDataPath}data/cell_x_gene_atlas.h5ad")

# Model training

In [ ]:
adata = sc.read(f"{processedDataPath}data/cell_x_gene_atlas.h5ad")

In [ ]:
# Setup scVI model using raw counts and batch info
scvi.model.SCVI.setup_anndata(
    adata,
    layer="counts",
    batch_key="batch",
)
model = scvi.model.SCVI(adata)

# Train scVI model
model.train()

In [ ]:
# Save data
if os.path.isdir(f"{processedDataPath}cell_x_gene_atlas_scVI_model"):
    shutil.rmtree(f"{processedDataPath}cell_x_gene_atlas_scVI_model")
model.save(f"{processedDataPath}cell_x_gene_atlas_scVI_model")

# Post processing

In [ ]:
adata = sc.read(f"{processedDataPath}data/cell_x_gene_atlas.h5ad")
model = scvi.model.SCVI.load(f"{processedDataPath}cell_x_gene_atlas_scVI_model", adata=adata)

In [ ]:
# get scVI latent space and normalized expression
adata.obsm["X_scVI"] = model.get_latent_representation()
sc.pp.neighbors(adata, use_rep="X_scVI")
sc.tl.tsne(adata, use_rep="X_scVI")
adata.layers["scVI_normalized"] = model.get_normalized_expression(library_size=1e4)

In [ ]:
# Save data
adata.write(f"{processedDataPath}data/cell_x_gene_atlas.h5ad")
adata.obs.to_csv(f"{processedDataPath}data/cell_x_gene_atlas_metadata.csv")
pd.DataFrame(adata.var_names).to_csv(f"{processedDataPath}data/cell_x_gene_atlas_genes.csv")
np.save(f"{processedDataPath}data/cell_x_gene_tsne.npy", adata.obsm["X_tsne"])

In [ ]:
# Filter on genes detected with at least 2 counts in at least 100 cells
exprs = np.log1p(adata.layers["counts"].todense())
exprs = pd.DataFrame(exprs, columns=adata.var_names)
num_cells_expressed = pd.DataFrame((exprs >= np.log1p(2)).sum())
sns.displot(num_cells_expressed[0], log_scale=True, kde=True)
plt.axvline(x=100, color="black")
plt.show()

In [ ]:
# Select genes to keep
keep_genes = num_cells_expressed[num_cells_expressed[0] >= 100].index
exprs = exprs[keep_genes]
np.save(f"{processedDataPath}expression/cell_x_gene_atlas_log1p_counts.npy", exprs)
keep_genes.to_csv(f"{processedDataPath}expression/keep_genes.csv")

In [ ]:
# Filter scVI normalized expression on same genes
exprs = adata.layers["scVI_normalized"]
exprs = pd.DataFrame(exprs, columns=adata.var_names)
exprs = exprs[keep_genes]

np.save(f"{processedDataPath}expression/cell_x_gene_atlas_expression.npy", exprs)

# TRA entropy scores

## scVI normalized

### Tissue

In [ ]:
exprs = np.load(f"{processedDataPath}expression/cell_x_gene_atlas_expression.npy")
genes = pd.read_csv(f"{processedDataPath}expression/keep_genes.csv")["feature_name"]
exprs = pd.DataFrame(exprs, columns=genes)
metadata = pd.read_csv(f"{processedDataPath}data/cell_x_gene_atlas_metadata.csv")

In [ ]:
exprs["tissue"] = metadata.tissue
exprs = exprs.groupby("tissue").mean()
np.save(f"{processedDataPath}expression/cell_x_gene_atlas_tissue_expression.npy", exprs)
pd.DataFrame(exprs.index).to_csv(f"{processedDataPath}expression/tissue_index.csv")
exprs = exprs**2
exprs = exprs/exprs.sum()
# Calculate tissue entropy for each gene
entropy = pd.DataFrame(
    exprs.apply(
        scipy.stats.entropy,
        axis=0,
        base=2
    ),
    columns=["entropy"]
)
entropy["entropy_score"] = 1-entropy.entropy/np.log2(exprs.shape[0])
entropy.to_csv(f"../../analysis/entropy/cell_x_gene_atlas_tissue_entropy.csv")

### Tissue (general)

In [ ]:
exprs = np.load(f"{processedDataPath}expression/cell_x_gene_atlas_expression.npy")
genes = pd.read_csv(f"{processedDataPath}expression/keep_genes.csv")["feature_name"]
exprs = pd.DataFrame(exprs, columns=genes)
metadata = pd.read_csv(f"{processedDataPath}data/cell_x_gene_atlas_metadata.csv")

In [ ]:
exprs["tissue_general"] = metadata.tissue_general
exprs = exprs.groupby("tissue_general").mean()
np.save(f"{processedDataPath}expression/cell_x_gene_atlas_tissue_general_expression.npy", exprs)
pd.DataFrame(exprs.index).to_csv(f"{processedDataPath}expression/tissue_general_index.csv")
exprs = exprs**2
exprs = exprs/exprs.sum()
# Calculate tissue entropy for each gene
entropy = pd.DataFrame(
    exprs.apply(
        scipy.stats.entropy,
        axis=0,
        base=2
    ),
    columns=["entropy"]
)
entropy["entropy_score"] = 1-entropy.entropy/np.log2(exprs.shape[0])
entropy.to_csv(f"../../analysis/entropy/cell_x_gene_atlas_tissue_general_entropy.csv")

### Cell type

In [ ]:
exprs = np.load(f"{processedDataPath}expression/cell_x_gene_atlas_expression.npy")
genes = pd.read_csv(f"{processedDataPath}expression/keep_genes.csv")["feature_name"]
exprs = pd.DataFrame(exprs, columns=genes)
metadata = pd.read_csv(f"{processedDataPath}data/cell_x_gene_atlas_metadata.csv")

In [ ]:
exprs["cell_type"] = metadata.cell_type
exprs = exprs.groupby("cell_type").mean()
np.save(f"{processedDataPath}expression/cell_x_gene_atlas_cell_type_expression.npy", exprs)
pd.DataFrame(exprs.index).to_csv(f"{processedDataPath}expression/cell_type_index.csv")
exprs = exprs**2
exprs = exprs/exprs.sum()
# Calculate tissue entropy for each gene
entropy = pd.DataFrame(
    exprs.apply(
        scipy.stats.entropy,
        axis=0,
        base=2
    ),
    columns=["entropy"]
)
entropy["entropy_score"] = 1-entropy.entropy/np.log2(exprs.shape[0])
entropy.to_csv(f"../../analysis/entropy/cell_x_gene_atlas_cell_type_entropy.csv")